In [313]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import warnings
warnings.filterwarnings("ignore")

In [314]:
stop_words = open(os.getcwd() + '/../Data/vietnamese-stopwords.txt', 'r', encoding='utf-8').read()
stop_words = stop_words.split()

In [315]:
url = 'https://phunuvietnam.vn/gan-1200-cong-nhan-o-tphcm-bi-sa-thai-truoc-tet-nhieu-nguoi-bat-khoc-trong-ngay-lam-viec-cuoi-cung-20221130194412875.htm'

result = requests.get(url)
c = result.content
soup = BeautifulSoup(c,"lxml")

summary = soup.find('div',class_="content-detail")
summary = summary.find_all('p')
summary = summary[1:-1]
summary = [i.text for i in summary]
summary = ' '.join(summary)
summary

'Đưa đôi tay quệt nước mắt, cô Nguyễn Thị Vốn (huyện Cao Lãnh, Đồng Tháp) cho biết 16 năm gắn bó với công ty, cô không nghĩ có một ngày, mình sẽ rời công ty như vậy. “Buồn lắm chứ, mà giờ công ty không có đơn hàng, biết phải làm sao. Hôm nay làm việc cuối cùng rồi, chắc cô về quê chứ đâu kiếm được việc nữa. Cô 52 tuổi rồi, ai mà mướn”, cô Vốn xúc động nói. Cũng trong tình cảnh tương tự, 17 năm gắn bó với công ty, hôm nay, cô Đặng Thị Hường (51 tuổi, quê Nghệ An) chỉ biết lắc đầu chua chát. Đẩy chiếc xe đạp cũ chở đầy ve chai rời công ty, cô Hường lủi thủi quay lại căn trọ cũ, cô cũng chưa biết ngày mai mình sẽ ra sao. Cô Đặng Thị Hường (51 tuổi, quê Nghệ An) 17 năm qua, số tiền 7-8 triệu mỗi tháng từ việc làm công nhân giúp cô trang trải sinh hoạt, gửi tiền về quê để chăm lo cho con cái, gia đình. Tuy cuộc sống không mấy khá giả nhưng có được công việc làm, đặc biệt ở tuổi ngoài 50 như cô đã là một điều may mắn. Cô tính ráng làm ít năm nữa, sau khi đủ tuổi về hưu, cô sẽ về lại quê nhà 

In [316]:
# Load modul 
import os
from gensim.models import KeyedVectors 

w2v = KeyedVectors.load_word2vec_format(os.getcwd() + '/../Data/vi.vec')

In [317]:
from underthesea import sent_tokenize,word_tokenize
#  convert sentence to vector / Hàm chuyển câu thành vector

def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(w2v[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

#  chuyển văn bản thành từng câu / Convert text to sentences
def text2sent(text):
    sentences = [[sentences.lower()] for sentences in sent_tokenize(text)]
    return sentences

#  tính độ tương tự giữa 2 câu / Calculate the similarity between 2 sentences
def similarity(s1, s2):
    return np.dot(s1, s2)

#  tính độ tương tự giữa 1 câu với tất cả các câu khác / Calculate the similarity between 1 sentence and all other sentences
def sim_matrix(sentences):
    n = len(sentences)
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            A[i][j] = similarity(sentences[i], sentences[j])
    return A


In [318]:
#  chuyển văn bản thành từng câu / Convert text to sentences
sentences = text2sent(summary)
sentences

[['đưa đôi tay quệt nước mắt, cô nguyễn thị vốn (huyện cao lãnh, đồng tháp) cho biết 16 năm gắn bó với công ty, cô không nghĩ có một ngày, mình sẽ rời công ty như vậy.'],
 ['“buồn lắm chứ, mà giờ công ty không có đơn hàng, biết phải làm sao.'],
 ['hôm nay làm việc cuối cùng rồi, chắc cô về quê chứ đâu kiếm được việc nữa.'],
 ['cô 52 tuổi rồi, ai mà mướn”, cô vốn xúc động nói.'],
 ['cũng trong tình cảnh tương tự, 17 năm gắn bó với công ty, hôm nay, cô đặng thị hường (51 tuổi, quê nghệ an) chỉ biết lắc đầu chua chát.'],
 ['đẩy chiếc xe đạp cũ chở đầy ve chai rời công ty, cô hường lủi thủi quay lại căn trọ cũ, cô cũng chưa biết ngày mai mình sẽ ra sao.'],
 ['cô đặng thị hường (51 tuổi, quê nghệ an) 17 năm qua, số tiền 7-8 triệu mỗi tháng từ việc làm công nhân giúp cô trang trải sinh hoạt, gửi tiền về quê để chăm lo cho con cái, gia đình.'],
 ['tuy cuộc sống không mấy khá giả nhưng có được công việc làm, đặc biệt ở tuổi ngoài 50 như cô đã là một điều may mắn.'],
 ['cô tính ráng làm ít năm 

In [319]:
#  Chuyển câu thành vector / Convert sentences to vector
sentences_vec = [sent2vec(i) for i in sentences]

# Tóm tắt văn bản thông thường theo cách tính trọng số

In [320]:
#  Tính ma trận độ tương tự / Calculate similarity matrix
A = sim_matrix(sentences_vec)

#  Tính độ tương tự trung bình của câu / Calculate the average similarity of the sentence
avg = A.sum(axis=1) / (A.shape[0] - 1)


In [321]:
#  Xây dựng trọng số cho từng câu / Build weight for each sentence
weighted_avg = ((avg - avg.min()) / (avg.max() - avg.min()))
#  Tính điểm cho từng câu / Calculate the score for each sentence
score = (A.dot(weighted_avg)) / weighted_avg.sum()
#  Sắp xếp câu theo thứ tự giảm dần / Sort sentences in descending order
ranked_sentences = sorted(((score[i],s) for i,s in enumerate(sentences_vec)), reverse=True)
    

In [322]:
#  In ra 5 câu có điểm cao nhất / Print the top 5 sentences
for i in range(5):
    print(ranked_sentences[i][1])

[ 0.07192806  0.11808205 -0.16493565 -0.01469135  0.02629197  0.0356265
 -0.10153046  0.00456999 -0.02225507  0.15692681 -0.08215572  0.0828931
 -0.03252504 -0.1402517  -0.06628973  0.02013916 -0.00569439 -0.06053371
  0.09176295 -0.14034715  0.005414    0.12056952 -0.16585721 -0.07193293
 -0.10832126  0.03510675  0.10341634 -0.00459006  0.09925169  0.02589221
 -0.05491049  0.0166328   0.05748543  0.0178882   0.0665076  -0.11608972
 -0.06865647  0.02791375 -0.02816372 -0.06491087  0.02607731  0.0158187
 -0.16718103 -0.03175634 -0.00373879 -0.26083565  0.04096742 -0.21112436
 -0.03348907  0.03410841 -0.02250595 -0.0477521   0.06043892 -0.11387746
  0.07111563 -0.19332562 -0.02735873  0.03363135  0.14613184 -0.18870847
  0.00617105  0.1578912   0.05940605  0.07700426 -0.09859898  0.09387803
 -0.08999343  0.12201154  0.03077221  0.03999867 -0.04504185 -0.16059135
 -0.23688263  0.13859442  0.09230885 -0.10851827 -0.00780156  0.06060214
  0.04074403  0.08765996  0.12685059  0.00608296 -0.03

In [323]:
#  Tạo ra văn bản tóm tắt với vector tương ứng / Create a summary text with the corresponding vector
summary_wv = ''
for i in range(5):
    summary_wv += sentences[np.where(sentences_vec == ranked_sentences[i][1])[0][0]][0] + ' '

print(summary_wv)


chị hường (40 tuổi, đồng tháp) vẫn chưa biết sẽ sống sao khi mất việc làm rời công ty với giỏ ve chai chất đầy trên yên xe, chị hường (40 tuổi, đồng tháp) cho biết sau ngày làm việc cuối cùng, hầu như ai cũng gom góp ít ve chai có được ở nơi sản xuất để ra vựa bán. chị trần thị giúp (48 tuổi) thẫn thờ lo lắng khi mất việc “ngày mai chị chưa biết sẽ làm gì, bình thường 7h sáng đã ra khỏi nhà để đi làm, tuy có cực nhưng có việc làm, có đồng tiền để xoay xở. trong căn nhà trọ, chị giúp cùng chồng lo lắng về cái tết cận kề “chắc tết năm nay buồn lắm, phải gói ghém tiết kiệm lại thôi, chứ giờ có mình ổng làm, hai vợ chồng cũng còn phải lo cho con cái rồi ông bà nội ngoại 2 bên nữa”, chị giúp nhìn chồng, thở dài bất lực. mọi dự định đều tan vỡ, cô hường chết lặng… “nay công ty cho nghỉ luôn rồi, chỉ có 1 số được ở lại thôi. với nhiều người, số tiền bán ve chai chẳng đáng là bao, nhưng với chị hường, được đồng nào hay đồng đó, nhất là những ngày sắp tới, chị chưa biết sẽ sống ra sao khi mất

# Sử dụng TFIDF và phương pháp cosine

In [324]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#  Tạo ma trận tf-idf / Create tf-idf matrix
vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform([s[0] for s in sentences])

#  Tính độ tương tự giữa các câu / Calculate the similarity between sentences
sim_mat = cosine_similarity(tf_idf_matrix, tf_idf_matrix)

#  Tính điểm cho từng câu / Calculate the score for each sentence
sim_scores = sim_mat.sum(axis=0)
ranked_sentences = sorted(((sim_scores[i],s) for i,s in enumerate(sentences)), reverse=True)

#  Chuyển 5 câu có điểm cao nhất thành văn bản tóm tắt / Convert the top 5 sentences to a summary text
summary_tfvec = ''
for i in range(5):
    summary_tfvec += ranked_sentences[i][1][0] + ' '

print(summary_tfvec)



chị hường (40 tuổi, đồng tháp) vẫn chưa biết sẽ sống sao khi mất việc làm rời công ty với giỏ ve chai chất đầy trên yên xe, chị hường (40 tuổi, đồng tháp) cho biết sau ngày làm việc cuối cùng, hầu như ai cũng gom góp ít ve chai có được ở nơi sản xuất để ra vựa bán. chị trần thị giúp (48 tuổi) thẫn thờ lo lắng khi mất việc “ngày mai chị chưa biết sẽ làm gì, bình thường 7h sáng đã ra khỏi nhà để đi làm, tuy có cực nhưng có việc làm, có đồng tiền để xoay xở. hôm nay làm việc cuối cùng rồi, chắc cô về quê chứ đâu kiếm được việc nữa. với nhiều người, số tiền bán ve chai chẳng đáng là bao, nhưng với chị hường, được đồng nào hay đồng đó, nhất là những ngày sắp tới, chị chưa biết sẽ sống ra sao khi mất việc làm. trong căn nhà trọ, chị giúp cùng chồng lo lắng về cái tết cận kề “chắc tết năm nay buồn lắm, phải gói ghém tiết kiệm lại thôi, chứ giờ có mình ổng làm, hai vợ chồng cũng còn phải lo cho con cái rồi ông bà nội ngoại 2 bên nữa”, chị giúp nhìn chồng, thở dài bất lực. 


# Sử dụng Count vector thay cho tfidfvector

In [325]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#  Tạo ma trận tf-idf / Create tf-idf matrix
vectorizer = CountVectorizer()
tf_idf_matrix = vectorizer.fit_transform([s[0] for s in sentences])

#  Tính độ tương tự giữa các câu / Calculate the similarity between sentences
sim_mat = cosine_similarity(tf_idf_matrix, tf_idf_matrix)

#  Tính điểm cho từng câu / Calculate the score for each sentence
sim_scores = sim_mat.sum(axis=0)
ranked_sentences = sorted(((sim_scores[i],s) for i,s in enumerate(sentences)), reverse=True)

#  Chuyển 5 câu có điểm cao nhất thành văn bản tóm tắt / Convert the top 5 sentences to a summary text
summary_countvec = ''
for i in range(5):
    summary_countvec += ranked_sentences[i][1][0] + ' '

print(summary_countvec)

chị hường (40 tuổi, đồng tháp) vẫn chưa biết sẽ sống sao khi mất việc làm rời công ty với giỏ ve chai chất đầy trên yên xe, chị hường (40 tuổi, đồng tháp) cho biết sau ngày làm việc cuối cùng, hầu như ai cũng gom góp ít ve chai có được ở nơi sản xuất để ra vựa bán. chị trần thị giúp (48 tuổi) thẫn thờ lo lắng khi mất việc “ngày mai chị chưa biết sẽ làm gì, bình thường 7h sáng đã ra khỏi nhà để đi làm, tuy có cực nhưng có việc làm, có đồng tiền để xoay xở. hôm nay làm việc cuối cùng rồi, chắc cô về quê chứ đâu kiếm được việc nữa. đưa đôi tay quệt nước mắt, cô nguyễn thị vốn (huyện cao lãnh, đồng tháp) cho biết 16 năm gắn bó với công ty, cô không nghĩ có một ngày, mình sẽ rời công ty như vậy. cô đặng thị hường (51 tuổi, quê nghệ an) 17 năm qua, số tiền 7-8 triệu mỗi tháng từ việc làm công nhân giúp cô trang trải sinh hoạt, gửi tiền về quê để chăm lo cho con cái, gia đình. 


# Sử dụng model kmeans dự đoán

In [326]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

#  Tạo ma trận tf-idf / Create tf-idf matrix
vectorizer_C = CountVectorizer()
tf_idf_matrix_C = vectorizer_C.fit_transform([s[0] for s in sentences])

#  Tạo model KMeans / Create KMeans model
kmeans = KMeans(n_clusters=5, random_state=0).fit(tf_idf_matrix_C)

#  Tìm ra các câu có cùng cluster / Find sentences with the same cluster
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, tf_idf_matrix_C)

#  Chuyển các câu có cùng cluster thành văn bản tóm tắt / Convert sentences with the same cluster to a summary text
summary_KM_C = ''
for i in closest:
    summary_KM_C += sentences[i][0] + ' '

print(summary_KM_C)

trong căn nhà trọ, chị giúp cùng chồng lo lắng về cái tết cận kề “chắc tết năm nay buồn lắm, phải gói ghém tiết kiệm lại thôi, chứ giờ có mình ổng làm, hai vợ chồng cũng còn phải lo cho con cái rồi ông bà nội ngoại 2 bên nữa”, chị giúp nhìn chồng, thở dài bất lực. mai chắc cô ở nhà, chưa biết làm gì. chị hường (40 tuổi, đồng tháp) vẫn chưa biết sẽ sống sao khi mất việc làm rời công ty với giỏ ve chai chất đầy trên yên xe, chị hường (40 tuổi, đồng tháp) cho biết sau ngày làm việc cuối cùng, hầu như ai cũng gom góp ít ve chai có được ở nơi sản xuất để ra vựa bán. chưa bao giờ, cuộc sống của những người công nhân lại khổ như vậy, nhất là tết đã cận kề đến nơi… hầu như ai cũng nhặt ve chai ở nơi sản xuất để ra vựa bán bởi họ biết tương lai sẽ đầy những khó khăn mai chắc cô ở nhà, chưa biết làm gì. 


In [327]:
#  Tạo ma trận tf-idf / Create tf-idf matrix
vectorizer_T = TfidfVectorizer()
tf_idf_matrix_T = vectorizer_T.fit_transform([s[0] for s in sentences])

#  Tạo model KMeans / Create KMeans model
kmeans = KMeans(n_clusters=5, random_state=0).fit(tf_idf_matrix_T)

#  Tìm ra các câu có cùng cluster / Find sentences with the same cluster
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, tf_idf_matrix_T)

#  Chuyển các câu có cùng cluster thành văn bản tóm tắt / Convert sentences with the same cluster to a summary text
summary_KM_T = ''
for i in closest:
    summary_KM_T += sentences[i][0] + ' '

print(summary_KM_T)


cũng trong tình cảnh tương tự, 17 năm gắn bó với công ty, hôm nay, cô đặng thị hường (51 tuổi, quê nghệ an) chỉ biết lắc đầu chua chát. chị hường (40 tuổi, đồng tháp) vẫn chưa biết sẽ sống sao khi mất việc làm rời công ty với giỏ ve chai chất đầy trên yên xe, chị hường (40 tuổi, đồng tháp) cho biết sau ngày làm việc cuối cùng, hầu như ai cũng gom góp ít ve chai có được ở nơi sản xuất để ra vựa bán. “cả ngày hôm nay chị buồn lắm, chắc thất nghiệp tới tết luôn rồi. giờ cô cũng không dám về quê nữa, tiền ít quá, lấy gì đâu mà về. cô cũng muốn xin đi rửa chén, phụ việc nhưng sợ mình lớn tuổi, cũng không ai nhận. 


In [328]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(sentences_vec)

#  Tìm ra các câu có cùng cluster / Find sentences with the same cluster
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sentences_vec)

#  Chuyển các câu có cùng cluster thành văn bản tóm tắt / Convert sentences with the same cluster to a summary text

summary_KM_WV = ''
for i in closest:
    summary_KM_WV += sentences[i][0] + ' '

print(summary_KM_WV)


trong số 1.185 công nhân của công ty tnhh tỷ hùng bị chấm dứt hợp đồng lao động, có đến 50% lao động là nữ ngoài 40 tuổi. mọi dự định đều tan vỡ, cô hường chết lặng… “nay công ty cho nghỉ luôn rồi, chỉ có 1 số được ở lại thôi. trong căn nhà trọ, chị giúp cùng chồng lo lắng về cái tết cận kề “chắc tết năm nay buồn lắm, phải gói ghém tiết kiệm lại thôi, chứ giờ có mình ổng làm, hai vợ chồng cũng còn phải lo cho con cái rồi ông bà nội ngoại 2 bên nữa”, chị giúp nhìn chồng, thở dài bất lực. chị hường (40 tuổi, đồng tháp) vẫn chưa biết sẽ sống sao khi mất việc làm rời công ty với giỏ ve chai chất đầy trên yên xe, chị hường (40 tuổi, đồng tháp) cho biết sau ngày làm việc cuối cùng, hầu như ai cũng gom góp ít ve chai có được ở nơi sản xuất để ra vựa bán. buồn lắm chứ, 17 năm rồi, đâu có nghĩ 1 ngày mình sẽ nghỉ việc như thế này đâu. 


# Thực hiện tóm tắt lại văn bản thành 1 câu với tất cả cá mô hình đã làm

In [331]:
# Chuyển các summeries thành dạng list / Convert summeries to list
summary_list = [summary_wv, summary_tfvec, summary_countvec, summary_KM_C, summary_KM_T, summary_KM_WV]

# Tách các câu trong các summeries / Split sentences in summeries
summary_list = [s.split('. ') for s in summary_list]

# Tạo ma trận tf-idf / Create tf-idf matrix
vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform([s[0] for s in sentences])

# Tính độ tương tự giữa các câu / Calculate the similarity between sentences
sim_mat = cosine_similarity(tf_idf_matrix, tf_idf_matrix)

# Tính điểm cho từng câu / Calculate the score for each sentence
sim_scores = sim_mat.sum(axis=0)

# Tạo một list chứa các câu có điểm cao nhất / Create a list containing the sentences with the highest score
ranked_sentences = sorted(((sim_scores[i],s) for i,s in enumerate(sentences)), reverse=True)

# Lấy câu có điểm cao nhất / Get the sentence with the highest score
top_sentence = ranked_sentences[0][1][0]
top_sentence

'chị hường (40 tuổi, đồng tháp) vẫn chưa biết sẽ sống sao khi mất việc làm rời công ty với giỏ ve chai chất đầy trên yên xe, chị hường (40 tuổi, đồng tháp) cho biết sau ngày làm việc cuối cùng, hầu như ai cũng gom góp ít ve chai có được ở nơi sản xuất để ra vựa bán.'

In [333]:
from underthesea import pos_tag
import string
# Xử lý câu có điểm cao nhất / Process the sentence with the highest score

# lấy các token của câu / get tokens of the sentence
top_sentence_tokens = word_tokenize(top_sentence)

# loại bỏ các token là dấu câu / remove punctuation tokens
top_sentence_tokens = [t for t in top_sentence_tokens if t not in string.punctuation]

# loại bỏ các token là stopword / remove stopword tokens
top_sentence_tokens = [t for t in top_sentence_tokens if t not in stop_words]

# Chuyển thành đoạn văn bản xúc tích / Convert to sentiment text
top_sentence_tokens = ' '.join(top_sentence_tokens)

print('Đoạn văn nói về: ', top_sentence_tokens)

Đoạn văn nói về:  chị hường 40 tuổi đồng tháp sống mất việc làm rời công ty giỏ ve chai chất đầy yên xe chị hường 40 tuổi đồng tháp làm việc cuối cùng hầu như gom góp ít ve chai sản xuất vựa bán
